In [289]:
import plotly.express as px
import numpy as np
import pandas as pd

In [290]:
trainers = pd.read_csv('../data/final_df.csv')

In [291]:
# print columns
print(trainers.columns)


Index(['type', 'gender', 'last_type', 'material', 'brand', 'sole_material',
       'lining_material', 'season', 'made_in', 'insole_material', 'name',
       'rating', 'code', 'cmr price', 'event price', 'full price',
       'discipline', 'interior_material'],
      dtype='object')


# Auditoría general de los datos

In [292]:
# mostrar porcentaje de valores nulos por columna
print(trainers.isnull().sum() / trainers.shape[0] * 100)

type                 10.863874
gender                0.163613
last_type             6.708115
material              0.654450
brand                11.812827
sole_material         2.977749
lining_material      23.985602
season               11.812827
made_in              12.009162
insole_material      26.047120
name                  0.000000
rating                0.000000
code                  0.000000
cmr price            68.553665
event price          68.553665
full price            0.000000
discipline           50.229058
interior_material    80.530105
dtype: float64


Se puede denotar que las columnas con mayor porcentaje de nulos son: 
- El tipo
- Marca
- Manterial de la suela interna, externa y material interior
- Disciplina


## Limpieza Columna Type

In [293]:
# muestra las columnas con valores nulos en tipo
trainers[trainers['type'].isnull()].head(2)


,type,gender,last_type,material,brand,sole_material,lining_material,season,made_in,insole_material,name,rating,code,cmr price,event price,full price,discipline,interior_material
3,NaN,Hombre,NaN,Textil,NaN,NaN,NaN,NaN,NaN,NaN,Tenis Hombre Reebok Rider V - Gris-Blanco,0.0,128281449,186459.0,2049.0,272900.0,NaN,NaN
4,NaN,Hombre,NaN,Cuero,NaN,Goma,NaN,NaN,NaN,NaN,Tenis New Balance 515-Verde,5.0,126747483,291191.0,31999.0,359990.0,Outdoor,NaN


In [294]:
trainers[trainers['type'].notnull()].head(2)

,type,gender,last_type,material,brand,sole_material,lining_material,season,made_in,insole_material,name,rating,code,cmr price,event price,full price,discipline,interior_material
0,Tenis moda,Niño,Normal,Cuero,reebok,Goma,Textil,Toda temporada,China,Textil,Tenis Reebok Classic para Niño,0.0,72793527,13989.0,15989.0,199900.0,NaN,NaN
1,Tenis moda,Niño,Normal,Cuero,reebok,Goma,Textil,Toda temporada,China,Textil,Tenis Reebok Classic para Niño,0.0,72793543,13989.0,15989.0,199900.0,NaN,NaN


considerando que existe una relación entre discipline y type podemos llenar los datos nulos considerando estas cateegorías. Por lo que se iniciará por limpiar la columna discipline de la cual se pueden inferir los valores faltantes de "Type"

In [295]:
trainers[trainers["discipline"].isnull()].head(2)

,type,gender,last_type,material,brand,sole_material,lining_material,season,made_in,insole_material,name,rating,code,cmr price,event price,full price,discipline,interior_material
0,Tenis moda,Niño,Normal,Cuero,reebok,Goma,Textil,Toda temporada,China,Textil,Tenis Reebok Classic para Niño,0.0,72793527,13989.0,15989.0,199900.0,NaN,NaN
1,Tenis moda,Niño,Normal,Cuero,reebok,Goma,Textil,Toda temporada,China,Textil,Tenis Reebok Classic para Niño,0.0,72793543,13989.0,15989.0,199900.0,NaN,NaN


In [296]:
t_moda = trainers[(trainers["type"] == "Tenis moda") & (trainers["discipline"].isnull())]
t_moda = t_moda.shape[0]
sin_disciplina = trainers[trainers["discipline"].isnull()]
sin_disciplina  = sin_disciplina.shape[0]

print("moda y sin disciplina: " + str(t_moda))
print("sin disciplina: " + str(sin_disciplina))

print("Porcentaje de moda y sin disciplina: " + str(t_moda / sin_disciplina * 100))

moda y sin disciplina: 1276
sin disciplina: 1535
Porcentaje de moda y sin disciplina: 83.1270358306189


El 83% de los tenis que no cuenta con disciplina es de moda. Por lo cual la categoría a asignar puede corresponer con aquella que tienen los tenis de moda.

In [297]:
f = trainers[(trainers["type"] == "Tenis moda") & (trainers["discipline"].notnull())]
f["discipline"].unique()

array(['Lifestyle'], dtype=object)

ya que todos los tenis con tipo tennis de moda es de "discipline" lifestyle hacemos la imputación.

In [298]:
# rellenar valores nulos de la columna discipline con "Lifestyle" cuando el tipo sea "Tenis moda"
trainers.loc[(trainers["type"] == "Tenis moda") & (trainers["discipline"].isnull()), "discipline"] = "Lifestyle"


In [299]:
trainers["discipline"].isnull().sum() /trainers.shape[0] * 100

8.475130890052355

se disminuyó el porcentaje de valores nulos en la columna discipline.

In [300]:
temp = trainers[(trainers["discipline"].isnull()) & (trainers["type"].notnull())]
len_temp = temp.shape[0]
per = len_temp / trainers.shape[0] * 100
per

4.810209424083769

In [301]:
temp["type"].unique()

array(['Guayos', 'Tenis deportivos', 'Zapatos casuales',
       'Calzado escolar', 'Tenis', 'Tenis con luces', 'Sandalias',
       'Tenis de fútbol', 'Botines', 'Mocasines'], dtype=object)

In [302]:
trainers = trainers.drop(trainers[trainers["type"] == "Sandalias"].index)
trainers = trainers.drop(trainers[trainers["type"] == "Botines"].index)
trainers = trainers.drop(trainers[trainers["type"] == "Mocasines"].index)
trainers = trainers.drop(trainers[trainers["type"] == "Calzado escolar"].index)


Estas no son variables necesarias para el análisis

In [303]:
temp["type"].unique()

array(['Guayos', 'Tenis deportivos', 'Zapatos casuales',
       'Calzado escolar', 'Tenis', 'Tenis con luces', 'Sandalias',
       'Tenis de fútbol', 'Botines', 'Mocasines'], dtype=object)

In [304]:
temp = trainers[(trainers["discipline"].isnull()) & (trainers["type"].notnull())]
len_temp = temp.shape[0]
per = len_temp / trainers.shape[0] * 100
per

4.625984251968504

In [305]:
trainers[trainers["type"] == "Zapatos casuales"].head(3)

,type,gender,last_type,material,brand,sole_material,lining_material,season,made_in,insole_material,name,rating,code,cmr price,event price,full price,discipline,interior_material
161,Zapatos casuales,Niño,Normal,Textil,NaN,Sintético (PU),Textil,NaN,NaN,NaN,Tenis Zanial Gris para Niño Los Gomosos,0.0,127062504,NaN,NaN,119900.0,NaN,NaN
277,Zapatos casuales,Niño,Normal,Textil,NaN,Sintético (PU),Textil,NaN,NaN,NaN,Tenis Kiquen Negro para Niño Los Gomosos,0.0,127039542,NaN,NaN,129900.0,NaN,NaN
410,Zapatos casuales,Niño,Normal,Sintético,NaN,Sintético (PU),Textil,NaN,NaN,NaN,Tenis Neywi Blanco-Azul O para Niño Los Gomosos,0.0,127062451,NaN,NaN,119900.0,NaN,NaN


# Visualizaciones para la limpieza

## Genero

In [306]:
gender_counts = trainers["gender"].value_counts()

fig = px.pie(trainers, names = "gender"
             )

fig.show()

Existen demasiadas categorias en la columna "type" y no se pueden agrupar en categorias mas generales, por lo que se eliminan las categorias que no son relevantes para el analisis.

In [307]:
trainers.loc[(trainers["gender"] == "Niño") | (trainers["gender"] == "Niña") | (trainers["gender"] == "Unisex niño") | (trainers["gender"] == "Unisex bebé") | (trainers["gender"] == "Bebé")  , "gender"] = "Infantil"

In [308]:
trainers.loc[(trainers["gender"] == "Unisex") | (trainers["gender"] == "Unisex adulto")  , "gender"] = "Unisex Adulto"

In [309]:
gender_counts = trainers["gender"].value_counts()

fig = px.pie(trainers, names = "gender"
             )

fig.show()

In [310]:
trainers[trainers["gender"].isnull()]

,type,gender,last_type,material,brand,sole_material,lining_material,season,made_in,insole_material,name,rating,code,cmr price,event price,full price,discipline,interior_material
981,NaN,NaN,NaN,NaN,NaN,Caucho,Malla (nylon),NaN,NaN,NaN,Asics Magic Speed 3 Tenis rojo de hombre para ...,0.0,128711940,NaN,NaN,849950.0,NaN,NaN
1163,NaN,NaN,Pequeña,Sintético,NaN,Sintético (PU),Textil,NaN,NaN,NaN,Tenis Casuales Azul-Café Bata Gotan Cor Hombre,0.0,126966902,NaN,NaN,159900.0,NaN,NaN
1196,NaN,NaN,NaN,Textil,NaN,Caucho,NaN,NaN,NaN,NaN,Nike Reactx Infinity Run 4 Tenis negro de homb...,0.0,127136851,NaN,NaN,837950.0,Running,NaN
1197,NaN,NaN,NaN,NaN,NaN,Caucho,Malla (nylon),NaN,NaN,NaN,Asics Gel-Contend 8 Tenis azul de hombre para ...,0.0,129422771,NaN,NaN,329950.0,NaN,NaN
1200,NaN,NaN,Pequeña,Textil,NaN,NaN,Sintético,NaN,NaN,NaN,Tenis Hombre Goodyear Negro DALLAS-A,0.0,120835945,NaN,NaN,249900.0,NaN,NaN


Todos los generos en Nan contienen en el nombre "hombre".

In [311]:
# rellenar valores nulos de la columna con Hombre 
trainers.loc[trainers["gender"].isnull(), "gender"] = "Hombre"

In [312]:
gender_counts = trainers["gender"].value_counts()

fig = px.pie(trainers, names = "gender"
             )

fig.show()

## Disciplina

In [338]:
df = trainers[["discipline", "type", "name"]]
df[ (df["type"].notnull())]

,discipline,type,name
0,Lifestyle,Tenis moda,Tenis Reebok Classic para Niño
1,Lifestyle,Tenis moda,Tenis Reebok Classic para Niño
2,Lifestyle,Tenis moda,Tenis moda Skechers Bounder Rozerg para Niño
6,Running,Tenis,Tenis Adidas Unisex Running Response Runner
7,Running,Tenis deportivos,Tenis Adidas para Hombre Running Supernova Stride
...,...,...,...
3051,Lifestyle,Tenis moda,Tenis New Balance para Hombre Moda 550
3052,Lifestyle,Tenis moda,Tenis Skechers para Mujer Moda Summits Perfect...
3053,Cross training,Tenis deportivos,Tenis Adidas Mujer Cross training Rapidmove Ad...
3054,Running,Tenis deportivos,Tenis Adidas para Hombre Running Adizero SL


In [335]:
df = trainers[["discipline", "type", "name"]]
df[(df["discipline"].isnull()) & (df["type"].notnull())]

,discipline,type,name
132,NaN,Guayos,Guayos Adidas Goletto VIII Infantil Unisex
157,NaN,Tenis deportivos,Tenis Adidas para Niño Advantage
161,NaN,Zapatos casuales,Tenis Zanial Gris para Niño Los Gomosos
277,NaN,Zapatos casuales,Tenis Kiquen Negro para Niño Los Gomosos
349,NaN,Guayos,Guayos Puma Rapido III Fg Niño
...,...,...,...
2939,NaN,Tenis con luces,Tenis con Luces Skechers Twinklesparksice Dre ...
3015,NaN,Tenis deportivos,Tenis deportivos Under Armour Surge 3 AC para ...
3020,NaN,Tenis deportivos,Tenis Diadora Aboltex para Niña Velcro
3026,NaN,Tenis deportivos,Tenis Starter Graf para Niño


In [343]:
trainers.loc[trainers["type"] == "Guayos", "discipline"] = "Fútbol"

In [342]:
df = trainers[["discipline", "type", "name"]]
df[(df["discipline"].isnull()) & (df["type"].notnull())]

,discipline,type,name
157,NaN,Tenis deportivos,Tenis Adidas para Niño Advantage
161,NaN,Zapatos casuales,Tenis Zanial Gris para Niño Los Gomosos
277,NaN,Zapatos casuales,Tenis Kiquen Negro para Niño Los Gomosos
396,NaN,Tenis,Tenis Adidas para Mujer Moda Urban Court
407,NaN,Tenis deportivos,Tenis Adidas para Niño Grand Court
...,...,...,...
2939,NaN,Tenis con luces,Tenis con Luces Skechers Twinklesparksice Dre ...
3015,NaN,Tenis deportivos,Tenis deportivos Under Armour Surge 3 AC para ...
3020,NaN,Tenis deportivos,Tenis Diadora Aboltex para Niña Velcro
3026,NaN,Tenis deportivos,Tenis Starter Graf para Niño


Reconociedo que el 80% de los tennis de la muestra son de running y lifestyle se podría enfocar el análisis en estas dos disciplinas particulares.

In [314]:
discipline_percentages = trainers['discipline'].value_counts(normalize=True) * 100
discipline_percentages

discipline
Lifestyle         62.075134
Running           27.334526
Fútbol             3.756708
Cross training     2.862254
Outdoor            1.538462
Training           0.894454
Básquetbol         0.644007
Tenis              0.250447
Trekking           0.214669
Walking            0.143113
Padel              0.071556
Trail running      0.071556
Ciclismo           0.071556
Baby fútbol        0.035778
Skate              0.035778
Name: proportion, dtype: float64

In [315]:
discipline_percentages = trainers['discipline'].value_counts(normalize=True) * 100

representative_disciplines = discipline_percentages[discipline_percentages > 5]

filtered_trainers = trainers[trainers['discipline'].isin(representative_disciplines.index)]

fig = px.bar(representative_disciplines, 
             x=representative_disciplines.index, 
             y=representative_disciplines.values,
             labels={'y': 'Percentage', 'x': 'Discipline'},
             title='Distribution of Disciplines among Trainers (Disciplines with more than 5% of the total)',
             color_discrete_sequence=['blue']
             )

# Mostrar el gráfico
fig.show()


Agr

# Visualizaciones analíticas

In [316]:
average_prices = filtered_trainers.groupby('made_in')['full price'].median().reset_index()

# Ordenar el DataFrame por precio promedio de manera descendente
average_prices_sorted = average_prices.sort_values(by='full price', ascending=False)

fig = px.bar(average_prices_sorted,
             x='made_in',  # Categorías en el eje x (país de fabricación)
             y='full price',  # Precio promedio en el eje y
             labels={'full_price': 'Average Price', 'made_in': 'Made In'},
             title='Average Price of Trainers by Country of Manufacture (Ordered by Average Price)',
             color='full price',  # Color basado en el precio promedio
             color_continuous_scale='greens'
             )

# Mostrar el gráfico
fig.show()

In [317]:
# correlacion entre made_in y full price

fig = px.box(filtered_trainers,
                    x='made_in',
                    y='full price',
                    title='Correlation between Country of Manufacture and Price',
                    labels={'full_price': 'Price', 'made_in': 'Made In'},
                    color='made_in',
                    color_discrete_sequence=px.colors.qualitative.Pastel
                    )

# Mostrar el gráfico
fig.show()